In [3]:
!pip install --upgrade pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 2.1 MB 26 kB/s eta 0:00:011
     |████████████████████████████████| 1.2 MB 4.0 MB/s eta 0:00:01
  Attempting uninstall: wheel
    Found existing installation: wheel 0.35.1
    Uninstalling wheel-0.35.1:
      Successfully uninstalled wheel-0.35.1
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.5.0
    Uninstalling setuptools-59.5.0:
      Successfully uninstalled setuptools-59.5.0
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient-utils 0.5.0 requires wheel<0.36.0,>=0.35.1, but you have wheel 0.37.1 which is incompatible.


In [4]:
!pip install indic_transliteration
!pip install contractions
!pip install sentencepiece
!pip install transformers
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 261.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.4/981.4 kB 80.1 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 110.5 kB/s eta 0:00:00 0:00:06m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 96.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 120.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.9/930.9 kB 150.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.0/359.0 kB 194.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 173.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 136.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import re
import time
import string
import contractions
import numpy as np
import pandas as pd

from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn import metrics

from transformers import XLMRobertaTokenizer, XLMRobertaModel, AdamW, get_linear_schedule_with_warmup

import matplotlib.pyplot as plt

In [2]:
import wandb

In [3]:
wandb.login()

wandb: Currently logged in as: alokpadhi. Use `wandb login --relogin` to force relogin


True

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
transliterate("yeh toh अद्भुत he", sanscript.ITRANS, sanscript.DEVANAGARI)

'येह् तोह् अद्भुत हे'

In [6]:
import json

In [7]:
with open("artifacts/run-3rxv0ijv-ProcessTrainDataframe:v0/Process Train Dataframe.table.json", 'r') as fp:
    train_data = json.load(fp)

In [8]:
train_df = pd.DataFrame(train_data["data"], columns=train_data["columns"])
train_df.head()

,Sentence,Sentiment,Sentiment Label
0,nen á vist bolest vztek smutek zmatek osam ě l...,neutral,2
1,Haan yaar neha 😔😔 kab karega woh post 😭 Usne n...,neutral,2
2,television media congress ke liye nhi h . Ye t...,negative,1
3,All India me nrc lagu kare w Kashmir se dhara ...,positive,0
4,Pagal hai kya ? They aren ’ t real issues Mand...,neutral,2


In [9]:
with open("artifacts/run-3rxv0ijv-ProcessTestDataframe:v0/Process Test Dataframe.table.json", 'r') as fp:
    test_data = json.load(fp)

In [10]:
test_df = pd.DataFrame(test_data["data"], columns=test_data["columns"])
test_df.head()

,Sentence,Sentiment,Sentiment Label
0,modi mantrimandal may samil honay par badhai n...,positive,0
1,Tu toh naamakool hai Mare h,negative,1
2,YOU saw caste and religion in them ... nation ...,negative,1
3,sir local police station pe complaint krne par...,neutral,2
4,Ve Maahi song from # Kesari is current favouri...,positive,0


In [11]:
from pathlib import Path

In [12]:
%ls

'Data Exploration and MLP.ipynb'   README.md           data/     wandb/
 LICENSE                           XLM-RoBERTa.ipynb   images/
 LSTM.ipynb                        artifacts/          model/


In [13]:
train_file = Path("data/train_14k_split_conll.txt")
test_file = Path("data/dev_3k_split_conll.txt")

In [14]:
# read the files
with open(train_file) as f:
    train_data = f.readlines()
    
with open(test_file, 'r') as f:
    test_data = f.readlines()

In [15]:
# Parse the data
def parse_data(data):
    uids, sentences, sentences_info, sentiment = [], [], [], []
    all_langs = []
    
    single_sentence, single_sentence_info = [], []
    sent = ""
    uid = 0
    
    for i, line in enumerate(data):
        line_ = line.strip()
        tokens = line_.split('\t')
        num_tokens = len(tokens)
        if num_tokens == 2:
            # add the word
            single_sentence.append(tokens[0])
            # add the language
            single_sentence_info.append(tokens[1])
            # all_langs.append(tokens[1])
        elif num_tokens == 3 and i > 0:
            sentences.append(single_sentence)
            sentences_info.append(single_sentence_info)
            sentiment.append(sent)
            uids.append(uid)
            sent = tokens[-1]
            uid = int(tokens[1])
            single_sentence = []
            single_sentence_info = []
        elif num_tokens == 1:
            continue
        else:
            sent = tokens[-1]
            uid = int(tokens[1])
            
    if len(single_sentence) > 0:
        sentences.append(single_sentence)
        sentences_info.append(single_sentence_info)
        sentiment.append(sent)
        uids.append(uid)
        
    assert len(sentences) == len(sentences_info) == len(sentiment) == len(uids)
    return sentences, sentences_info, sentiment, uids

In [16]:
sentences, sentences_info, sentiment, uids = parse_data(train_data)

In [17]:
test_sentences, test_sentences_info, test_sentiment, test_uids = parse_data(test_data)

In [18]:
def translate(sentences, sentences_info):
    translated_sentneces = []
    for sent, sent_info in zip(sentences, sentences_info):
        translated_sentence = []
        for word, word_info in zip(sent, sent_info):
            if word_info == "Hin":
                translated_sentence.append(transliterate(word, sanscript.ITRANS, sanscript.DEVANAGARI))
            else:
                translated_sentence.append(word)
        translated_sentneces.append(translated_sentence)
    return translated_sentneces

In [19]:
translated_sentences = translate(sentences, sentences_info)
test_translated_sentences = translate(test_sentences, test_sentences_info)

In [20]:
# Data Preprocessing
url_pattern = r'https(.*)/\s\w+'
names_with_numbers = r'([A-Za-z]+)\d{3,}'
apostee = r"([\w]+)\s'\s([\w]+)"
names = r"@[\s]*[\w]+[\s]*[_]+[\s]*[\w]+|@[\s]*[\w]+"

def preprocess_data(sentence_tokens):
    sentence = " ".join(sentence_tokens)
    sentence = " " + sentence
    
    # Remove RT and ...
    sentence = re.sub("\sRT\s", "", sentence)
    sentence = sentence.replace("…", "")
    sentence = re.sub(re.compile(names), " ", sentence)
    sentence = re.sub(re.compile(url_pattern), "", sentence)
    sentence = re.sub(re.compile(apostee), r"\1'\2", sentence)
    
    # fixing contractions
    sentence = contractions.fix(sentence)
    
    sentence = re.sub(re.compile(names_with_numbers), r" ", sentence)
    
    sentence = " ".join(sentence.split()).strip()
    return sentence

In [21]:
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

In [22]:
xlmr_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
# preprocess all training samples
processed_sentences = []
for sent in sentences:
    processed_sentences.append(preprocess_data(sent))

In [24]:
test_processed_sentences = []
for sent in test_sentences:
    test_processed_sentences.append(preprocess_data(sent))

In [25]:
sentiment_mapping = {
    "positive": 0,
    "negative": 1,
    "neutral": 2
}

In [26]:
labels = [sentiment_mapping[sent] for sent in sentiment]
test_labels = [sentiment_mapping[sent] for sent in test_sentiment]

In [27]:
train_uids, val_uids, train_X, val_X, train_y, val_y = train_test_split(uids, processed_sentences, labels, test_size=0.25)

In [28]:
train_token_lengths = [len(sent.split()) for sent in train_X]

In [29]:
MAX_LEN = max(train_token_lengths)

In [30]:
class HinglishDataset(Dataset):
    def __init__(self, inputs, labels, tokenizer, max_len):
        self.sentences = inputs
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        sentiment = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return {
            "text": sentence,
            "input_ids": encoding['input_ids'].flatten(),
            "attention_mask": encoding['attention_mask'].flatten(),
            "label": torch.tensor(sentiment, dtype=torch.long)
        }
    
    def create_dataloader(self, batch_size, shuffle=False):
        return DataLoader(dataset=self, shuffle=shuffle, batch_size=batch_size)

In [31]:
train_dataset = HinglishDataset(train_X, train_y, tokenizer, MAX_LEN)
val_dataset = HinglishDataset(val_X, val_y, tokenizer, MAX_LEN)
test_dataset = HinglishDataset(test_processed_sentences, test_labels, tokenizer, MAX_LEN)

In [32]:
batch_size=64

In [33]:
train_dataloader = train_dataset.create_dataloader(batch_size=batch_size, shuffle=True)
val_dataloader = val_dataset.create_dataloader(batch_size=batch_size)
test_dataloader = test_dataset.create_dataloader(batch_size=batch_size)                                                   

In [34]:
xlmr_model

XLMRobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): 

In [35]:
xlmr_model.config

XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

In [36]:
class XLMModel(nn.Module):
    def __init__(self, output_dim, xlmr_model, dropout=0.5):
        super().__init__()
        
        self.xlmr = xlmr_model
        hidden_size = self.xlmr.config.to_dict()['hidden_size']
        
        self.out = nn.Linear(hidden_size, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input_ids, attention_mask):
        xlmr_output = self.xlmr(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        # print(pooled_output)
        logits = self.out(self.dropout(xlmr_output.pooler_output))
        
        return logits        

In [37]:
output_dim = 3
xlmr_model = xlmr_model.to(device)
model = XLMModel(output_dim, xlmr_model)

In [38]:
model = model.to(device)

In [39]:
epochs = 5
lr = 3e-5
min_lr = 5e-6
lr_decay = 0.5
lr_patience = 2

In [40]:
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', lr_decay, lr_patience, verbose=True, min_lr=min_lr)
loss_fn = nn.CrossEntropyLoss().to(device)

In [41]:
def train(model, iterator, clip=2.0):
    epoch_loss = 0
    model.train()
    
    for batch in iterator:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        targets = batch["label"].to(device)
        
        predictions = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        optimizer.zero_grad()
        loss = loss_fn(predictions, targets)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss/len(iterator)

In [42]:
def simple_accuracy(preds, labels):
    return np.equal(preds, labels).mean()

In [43]:
def evaluate(model, iterator):
    model.eval()
    epoch_loss = 0
    preds = []
    targets = []
    
    with torch.no_grad():
        for batch in iterator:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)
            
            predictions = model(
                input_ids,
                attention_mask
            )
            
            loss = loss_fn(predictions, labels)
            epoch_loss += loss.item()
            
            targets.extend(labels.detach().cpu().numpy().tolist())
            _, predicted = torch.max(predictions, 1)
            preds.extend(predicted.detach().cpu().numpy().tolist())
            
        return epoch_loss/len(iterator), simple_accuracy(preds, targets)

In [44]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    
    return elapsed_mins, elapsed_secs

In [45]:
config = dict(
    batch_size=batch_size,
    epochs=epochs,
    learning_rate=lr,
    scheduler=scheduler,
    weight_decay=lr_decay,
    patience=lr_patience,
    device=device,
    model_name="XLMRoBERTaBase"
)

In [46]:
xlm_run = wandb.init(project="Sentiment Analysis-Hinglish", config=config, name="XLM_RoBERTa")

In [47]:
model_path = "model/xlm_model.pt"

In [48]:
best_valid_loss = float('inf')
wandb.watch(model, log_freq=100)
for epoch in range(epochs):
    start_time = time.time()
    print(f"[INFO] Starting of Training for Epoch: {epoch+ 1:.2f}")
    train_loss = train(model, train_dataloader)
    print(f"[INFO] Finished Training for Epoch: {epoch+1:.2f}")
    print(f"[INFO] Starting of Validating for Epoch: {epoch+1:.2f}")
    val_loss, val_acc = evaluate(model, val_dataloader)
    print(f"[INFO] Validation for Epoch: {epoch+1:.2f} finished.")
    end_time = time.time()
    
    
    wandb.log({"train_loss": train_loss})
    wandb.log({"val_loss": val_loss})
    wandb.log({"val_accuracy": val_acc})
    scheduler.step(val_loss)
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f"Epoch: {epoch + 1:02} | Time Taken: {epoch_mins}mins:{epoch_secs:.2f}secs")
    print(f"Train loss: {train_loss:.3f} | val_loss: {val_loss:.3f} | val_accuracy:{val_acc:.2f}")
    
    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        xlm_run.summary["Best validation Loss"] = best_valid_loss
        torch.save(model.state_dict(), model_path)
        model_artifact = wandb.Artifact("XLMRoBERTaModel", type="model")
        model_artifact.add_file(model_path)
        xlm_run.log_artifact(model_artifact)

[INFO] Starting of Training for Epoch: 1.00


/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


[INFO] Finished Training for Epoch: 1.00
[INFO] Starting of Validating for Epoch: 1.00
[INFO] Validation for Epoch: 1.00 finished.
Epoch: 01 | Time Taken: 1mins:28.00secs
Train loss: 0.964 | val_loss: 0.820 | val_accuracy:0.63
[INFO] Starting of Training for Epoch: 2.00
[INFO] Finished Training for Epoch: 2.00
[INFO] Starting of Validating for Epoch: 2.00
[INFO] Validation for Epoch: 2.00 finished.
Epoch: 02 | Time Taken: 1mins:29.00secs
Train loss: 0.834 | val_loss: 0.895 | val_accuracy:0.59
[INFO] Starting of Training for Epoch: 3.00
[INFO] Finished Training for Epoch: 3.00
[INFO] Starting of Validating for Epoch: 3.00
[INFO] Validation for Epoch: 3.00 finished.
Epoch: 03 | Time Taken: 1mins:27.00secs
Train loss: 0.758 | val_loss: 0.808 | val_accuracy:0.64
[INFO] Starting of Training for Epoch: 4.00
[INFO] Finished Training for Epoch: 4.00
[INFO] Starting of Validating for Epoch: 4.00
[INFO] Validation for Epoch: 4.00 finished.
Epoch: 04 | Time Taken: 1mins:29.00secs
Train loss: 0.68

In [67]:
inputs = tokenizer("this is", return_tensors="pt")

In [69]:
inputs = inputs.to(device)

In [70]:
outputs = xlmr_model(**inputs)

In [72]:
outputs['last_hidden_state'].shape

torch.Size([1, 4, 768])

In [51]:
with torch.no_grad():
    model.eval()
    preds = []
    trgs = []
    for batch in test_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        targets = batch["label"].to(device)
        
        outputs = model(
            input_ids, attention_mask
        )
        
        _, predicted = torch.max(outputs, 1)
        preds.extend(predicted.detach().cpu().numpy().tolist())
        trgs.extend(targets.detach().cpu().numpy().tolist())

In [57]:
print(metrics.classification_report(trgs, preds))

              precision    recall  f1-score   support

           0       0.63      0.76      0.69       982
           1       0.62      0.72      0.66       890
           2       0.61      0.41      0.49      1128

    accuracy                           0.62      3000
   macro avg       0.62      0.63      0.61      3000
weighted avg       0.62      0.62      0.61      3000



In [59]:
performance = metrics.precision_recall_fscore_support(trgs, preds, average="weighted")

In [61]:
wandb.log({"precision": performance[0], "recall": performance[1], "f1-score": performance[2]})